Homework 2: Discovery of Frequent Itemsets and Association Rules

In [126]:
# Discovery of Frequent Itemsets and Association Rules

# read dataset from T10I4D100K.dat
# read intergers sets from dataset line by line
def read_dataset(filename):
    dataset = []
    with open(filename, 'r') as f:
        for line in f:
            lines = list(map(int, line.strip().split()))
            # lines.sort()
            dataset.append(lines)
    # dataset.sort()
    return dataset

dataset = read_dataset('./T10I4D100K.dat')
# length of dataset
print('length of dataset:', len(dataset))

length of dataset: 100000


implement the A-Priori algorithm for finding frequent itemsets with support at least s in a dataset of sales transactions.

In [127]:
# get the apporopriate support threshold
# typicallly, support threshold is the length of dataset / 100
def get_support_threshold(dataset):
    return len(dataset) * 0.01

print('support threshold:', int(get_support_threshold(dataset)))

s = get_support_threshold(dataset)
# s = 3000

support threshold: 1000


In [129]:
# finding frequent itemsets of integers with support at least s in the dataset
import numpy as np
import pandas as pd
import itertools
class Apriori:
    def __init__(self, dataset, support_threshold):
        self.dataset = dataset
        self.support_threshold = support_threshold
        self.frequent_itemsets = []
        self.frequent_item = []
        self.all_freq_itemsets = []
        self.item_frequency = {}
        self.frequent_item_sets = []
        
   # compute the support of given itemset
    def support(self, itemset):
        support = 0
        for transaction in self.dataset:
            if set(itemset).issubset(set(transaction)):
                support += 1
        return support
    
    # filter generated possible itemset
    def filter_itemset(self, itemset):
        return (self.support(itemset) >= self.support_threshold)

    # get 1-item_set
    def get_1_item_set(self):
        for transaction in self.dataset:
            for item in transaction:
                if item not in self.item_frequency:
                    self.item_frequency[item] = 1
                else:
                    self.item_frequency[item] += 1
        for item in self.item_frequency:
            if self.item_frequency[item] >= self.support_threshold:
                self.frequent_item.append(item)
        # self.frequent_item.sort()
        return self.frequent_item

    # update the dataset
    def update_dataset(self, itemset):
        new_dataset = []
        for transaction in self.dataset:
            for item in transaction:
                if item in itemset:
                    new_dataset.append(transaction)
                    break
        # new_dataset.sort()
        self.dataset = new_dataset
        return self.dataset
    
    # get all k size subsets contained given itemset of given transactions
    def get_k_size_subsets(self, itemset, transactions, k):
        return [sets for sets in itertools.combinations(transactions, k) if sets in itemset]
    
    # get k-item_set
    # compute the support using the support function
    # def get_k_item_set(self, _k_1_itemsets, k):
    #     k_itemsets = []
    #     # get all elements in the _k_1_itemsets
    #     all_elements = np.hstack(np.array(_k_1_itemsets))
    #     # get all unique elements in the _k_1_itemsets
    #     unique_elements = np.unique(all_elements)
    #     # get all combinations of unique elements
    #     possible_itemsets = list(itertools.combinations(unique_elements, k))
    #     print(len(possible_itemsets))
    #     # possible_itemsets.sort()
    #     for itemset in possible_itemsets:
    #         if self.filter_itemset(itemset):
    #             k_itemsets.append(itemset)
    #             print(f'{k}-itemset:', itemset, 'support:', self.support(itemset))
    #     return k_itemsets
    def get_k_item_set(self, _k_1_itemsets, k):
        k_itemsets = []
        # get all elements in the _k_1_itemsets
        all_elements = np.hstack(np.array(_k_1_itemsets))
        # get all unique elements in the _k_1_itemsets
        unique_elements = np.unique(all_elements)
        # get all combinations of unique elements
        possible_itemsets = list(itertools.combinations(unique_elements, k))
        # convert list to dict and set all values to 0
        possible_itemsets = {itemset: 0 for itemset in possible_itemsets}
        # print(len(possible_itemsets))
        # possible_itemsets.sort()
        for transaction in self.dataset:
            subsets = self.get_k_size_subsets(possible_itemsets, transaction, k)
            for s in subsets:
                possible_itemsets[s] += 1
        k_itemsets = [item for item in possible_itemsets if possible_itemsets[item] >= self.support_threshold]
        print(f'{k}-itemset:', k_itemsets,)
        # for itemset in possible_itemsets:
        #     if self.filter_itemset(itemset):
        #         k_itemsets.append(itemset)
        #         print(f'{k}-itemset:', itemset, 'support:', self.support(itemset))
        return k_itemsets

    # get frequent itemset
    def get_frequent_itemset(self):
        _1_itemsets = self.get_1_item_set()
        self.update_dataset(_1_itemsets)
        # print(len(_1_itemsets))
        self.frequent_itemsets.append(_1_itemsets)
        k = 2
        while True:
            k_itemsets = self.get_k_item_set(_1_itemsets, k)
            if len(k_itemsets) == 0:
                break
            self.frequent_itemsets.append(k_itemsets)
            _1_itemsets = k_itemsets
            self.update_dataset_by_deleting_infrequent_items(k_itemsets)
            k += 1
        return self.frequent_itemsets

    # update the dataset by deleting the infrequent items
    def update_dataset_by_deleting_infrequent_items(self, itemsets):
        new_dataset = []
        for transaction in self.dataset:
            for itemset in itemsets:
                if set(itemset).issubset(set(transaction)):
                    new_dataset.append(transaction)
                    break
        self.dataset = new_dataset
        return self.dataset
    
# test
apriori = Apriori(dataset, s)
fre_items = apriori.get_frequent_itemset()
print(fre_items)
print(len(fre_items))
print(fre_items[0], fre_items[1], fre_items[2])



70125


KeyboardInterrupt: 

In [ ]:
# frequent_items = [(39, 704), (39, 825), (217, 346), (227, 390), (368, 682), (368, 829), (390, 722), (704, 825), (789, 829), (39, 704, 825)]
frequent_items = fre_items[1:]
# get the association rules
def get_association_rules(frequent_itemsets):
    rules = []
    apriori = Apriori(dataset, s)
    for itemset in frequent_itemsets:
        for item in itemset:
            for item_subset in list(itertools.combinations(itemset, len(itemset) - 1)):
                confidence = apriori.support(itemset) / apriori.support(item_subset)
                if item not in item_subset:
                    rules.append((item_subset, item, confidence))
    return rules
#test 
rules = []
rules = get_association_rules(frequent_items)
print(rules)
print(len(rules))
# print rules in the form of: item => item confidence
for rule in rules:
    print(set(rule[0]), '=>', rule[1], 'confidence:', rule[2])


[((704,), 39, 0.617056856187291), ((39,), 704, 0.259981211836543), ((825,), 39, 0.3847649918962723), ((39,), 825, 0.27876937529356505), ((346,), 217, 0.385014409221902), ((217,), 346, 0.24855813953488373), ((390,), 227, 0.3906890130353817), ((227,), 390, 0.577007700770077), ((682,), 368, 0.2887221684414327), ((368,), 682, 0.15240163515585078), ((829,), 368, 0.17533039647577092), ((368,), 829, 0.15252938170669392), ((722,), 390, 0.17827202737382378), ((390,), 722, 0.3880819366852886), ((825,), 704, 0.35721231766612643), ((704,), 825, 0.6142697881828316), ((829,), 789, 0.17533039647577092), ((789,), 829, 0.27709445346948247), ((704, 825), 39, 0.9392014519056261), ((39, 825), 704, 0.8719460825610783), ((39, 704), 825, 0.9349593495934959)]
21
{704} => 39 confidence: 0.617056856187291
{39} => 704 confidence: 0.259981211836543
{825} => 39 confidence: 0.3847649918962723
{39} => 825 confidence: 0.27876937529356505
{346} => 217 confidence: 0.385014409221902
{217} => 346 confidence: 0.2485581395